In [4]:
%matplotlib inline

import os
import pandas as pd
import xgboost as xgb
from sklearn import metrics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pickle
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
import pandas as pd

In [5]:
# load test data 
dfp = '/home/jovyan/work/ngr4/data/ed_data/final200427/final_4_27'
pdfp = '/home/jovyan/work/ngr4/data/processed/'
pfp = '/home/jovyan/work/ngr4/results/'

X_test = pd.read_csv(os.path.join(pdfp,'X_test.csv'),index_col=0)
y_test = pd.read_csv(os.path.join(pdfp,'y_test.csv'),index_col=0)
X_test_cci = pd.read_csv(os.path.join(pdfp,'X_test_cci.csv'),index_col=0)




In [6]:
def qcsi(df):
    """Calcualate quick-CSI and return score for training.
    Arguments:
        df (pd.DataFrame): can be within cv-fold
    """
    # impute
    temp = SimpleImputer(verbose=1, strategy='median').fit_transform(df)
    ## convert back to pandas
    df = pd.DataFrame(temp, columns=df.columns, dtype='float64')
    temp = pd.DataFrame()
    temp['last_o2_flow'] = pd.cut(df['last_o2_flow'],
                                 bins=[-np.inf,2,4,np.inf],
                                 labels=['no_nc','2-4','gt4'])
    temp['min_SPO2'] = pd.cut(df['min_SPO2'],
                                 bins=[-np.inf,88,92,np.inf],
                                 labels=['lt88','88-92','gt92'])
    temp['last_RR'] = pd.cut(df['last_RR'],
                                 bins=[-np.inf,22,28,np.inf],
                                 labels=['lt22','22-28','gt28'])
    # round(2*OR) - 2
    temp['qcsi'] = 0
    temp.loc[temp['last_o2_flow']=='2-4','qcsi'] = 4 + temp.loc[temp['last_o2_flow']=='2-4','qcsi']
    temp.loc[temp['last_o2_flow']=='gt4','qcsi'] = 5 + temp.loc[temp['last_o2_flow']=='gt4','qcsi']
    temp.loc[temp['min_SPO2']=='lt88','qcsi'] = 5 + temp.loc[temp['min_SPO2']=='lt88','qcsi']
    temp.loc[temp['min_SPO2']=='88-92','qcsi'] = 2 + temp.loc[temp['min_SPO2']=='88-92','qcsi']
    temp.loc[temp['last_RR']=='22-28','qcsi'] = 1 + temp.loc[temp['last_RR']=='22-28','qcsi']
    temp.loc[temp['last_RR']=='gt28','qcsi'] = 2 + temp.loc[temp['last_RR']=='gt28','qcsi']
    
    return temp['qcsi'] 

def proper_curb65(df):
    """CURB-65 with consistency for qcsi.
    
    Now corrected for GCS < 15
    
    Arguments:
        df (pd.DataFrame): can be within cv-fold
        
    Returns:
        pd.Series
    """
    # impute
    temp = SimpleImputer(verbose=1, strategy='median').fit_transform(df)
    
    ## convert back to pandas
    df = pd.DataFrame(temp, columns=df.columns, dtype='float64')
    
    df['curb65'] = (df['last_GCS']<15).astype(int) + (df['bun']>=19).astype(int) + (df['last_RR']>=30).astype(int) + ((df['last_SBP']<90)|(df['last_DBP']<=60)).astype(int) + (df['age']>=65)
    return df['curb65']

def proper_qSOFA(df):
    """Calculate qSOFA score based on worst measurements.
    
    1-pt for each, GCS=<15, RR≥22, sBP<=100.
    
    Returns:
        pd.DataFrame: with value added into col='curb65'
    
    Reference:
        https://www.mdcalc.com/qsofa-quick-sofa-score-sepsis
    """
    # impute
    temp = SimpleImputer(verbose=1, strategy='median').fit_transform(df)
    
    ## convert back to pandas
    df = pd.DataFrame(temp, columns=df.columns, dtype='float64')
    df['qSOFA'] = (df['last_GCS']<15).astype(int) + (df['last_RR']>=22).astype(int) + (df['last_SBP']<=100)
    return df['qSOFA']

def pp_X(df):
    min_feat = ['min_SPO2','last_o2_flow','last_RR','bun',
                'ast','age','last_SBP',
                'glucose',
                'WBC','PROCAL','FERRITIN','CRP',
                'creatinine','chloride','alt']
    if False:
        df.loc[df['ox_status240_ROOM_AIR']==1,'last_o2_flow'] = 0 
    df = df.loc[:,min_feat]
    return df

def pp_qcsi(df):
    min_feat = ['min_SPO2','last_o2_flow','last_RR']
    if True:
        df.loc[df['ox_status240_ROOM_AIR']==1,'last_o2_flow'] = 0 
    df = df.loc[:,min_feat]
    return df

def pp_cci(df):
    # impute
    temp = SimpleImputer(verbose=1, strategy='median').fit_transform(df)
    
    ## convert back to pandas
    df = pd.DataFrame(temp, columns=df.columns, dtype='float64')
    
    return df['mrtlt_scr']
    

In [11]:
models = ['curb65redo','qSOFAredo','qCSIredo','CSIredo','cci']
methods = ['LR','LR','LR','XGB','LR']

results = pd.DataFrame()

for model_name,method in zip(models,methods):
    
    model_pkl = glob.glob(os.path.join(pdfp,'{}_{}iter*.pkl'.format(model_name,method)))[0]
    model = pickle.load(open(model_pkl, "rb"))   
    
    if method=='XGB':
        dtest = xgb.DMatrix(pp_X(X_test), label=y_test)
        p1 = model.predict(dtest, ntree_limit=model.best_iteration)
        short_name = 'CSI'
    elif method=='LR' and 'qCSI' in model_name:
        p1 = model.predict_proba(qcsi(pp_qcsi(X_test)).to_numpy().reshape(-1,1))[:,1]
        short_name = 'qCSI'
    elif method=='LR' and 'qSOFA' in model_name:
        p1 = model.predict_proba(proper_qSOFA(X_test).to_numpy().reshape(-1,1))[:,1]
        short_name = 'qSOFA'
    elif method=='LR' and 'curb65' in model_name:
        p1 = model.predict_proba(proper_curb65(X_test).to_numpy().reshape(-1,1))[:,1]
        short_name = 'CURB-65'
    elif method=='LR' and 'cci' in model_name:
        p1 = model.predict_proba(pp_cci(X_test_cci).to_numpy().reshape(-1,1))[:,1]
        short_name = 'Elixhauser'
    
    # metrics
    fpr, tpr, thresholds = metrics.roc_curve(y_test, p1)
    optimal_idx = np.argmax(tpr-fpr)
    optimal_threshold = thresholds[optimal_idx]
    optimal_pred = (p1>optimal_threshold).astype(int)
    precision,recall,_ = metrics.precision_recall_curve(y_test, p1)
    auprc = metrics.auc(recall, precision)
    auroc = metrics.roc_auc_score(y_test,p1)
    ap = metrics.average_precision_score(y_test,p1)
    bs = metrics.brier_score_loss(y_test,p1)
    f1 = metrics.f1_score(y_test,optimal_pred)
    acc = metrics.accuracy_score(y_test,optimal_pred)

    # results
    print('\n{} {} results'.format(model_name, method))
    print('---------------------------------')
    print('Accuracy:          {:.4f}%'.format(acc))
    print('Sensitivity:       {:.4f}'.format(tpr[optimal_idx]))
    print('Specificity:       {:.4f}'.format(1-fpr[optimal_idx]))
    print('AU-ROC:            {:.4f}'.format(auroc))
    print('AU-PRC:            {:.4f}'.format(auprc))
    print('Average-precision: {:.4f}'.format(ap))
    print('Brier score:       {:.4f}'.format(bs))
    print('F1-score:          {:.4f}'.format(f1))
    
    dt = pd.DataFrame({'':short_name,
                       'AU-ROC':auroc,
                       'Accuracy':acc,
                       'Sensitivity':tpr[optimal_idx],
                       'Specificity':1-fpr[optimal_idx],
                       'AU-PRC':auprc,
                       'Brier score':bs,
                       'F1':f1,
                       'Average Precision':ap},index=[short_name])
    results = results.append(dt, ignore_index=True)
results.to_csv(os.path.join(pdfp,'test_eval.csv'))



curb65redo LR results
---------------------------------
Accuracy:          0.4708%
Sensitivity:       0.8485
Specificity:       0.2222
AU-ROC:            0.5015
AU-PRC:            0.1793
Average-precision: 0.1503
Brier score:       0.1187
F1-score:          0.1911

qSOFAredo LR results
---------------------------------
Accuracy:          0.8333%
Sensitivity:       0.4848
Specificity:       0.7005
AU-ROC:            0.5919
AU-PRC:            0.2185
Average-precision: 0.1937
Brier score:       0.1166
F1-score:          0.0909

qCSIredo LR results
---------------------------------
Accuracy:          0.8208%
Sensitivity:       0.7879
Specificity:       0.7778
AU-ROC:            0.8114
AU-PRC:            0.4633
Average-precision: 0.4299
Brier score:       0.0975
F1-score:          0.4941

CSIredo XGB results
---------------------------------
Accuracy:          0.7750%
Sensitivity:       0.7273
Specificity:       0.7874
AU-ROC:            0.7618
AU-PRC:            0.4005
Average-precision: 

In [13]:
results.to_csv(os.path.join(pdfp,'test_eval.csv'))